In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
import datetime
import os
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker


#Connect to DB
path = os.path.abspath("MinorLeague.db")
Base = declarative_base()
engine = create_engine("sqlite:///"+path, echo = False)
conn = engine.connect()
metadata = Base.metadata
Session = sessionmaker(bind=engine)
session = Session()

#Read in the data
df = pd.read_sql_table('all_3_Data', conn)

#To remove any columns from the data
#df = df.drop(['StateCode'], axis = 1)
#df = df.drop(['CountyCode'], axis = 1)
df = df.drop(['meanIncome'], axis = 1)

#Convert the Year column to a datetime datatype
df['Year'] = pd.to_datetime(df.Year, format='%Y')

#Convert the StateCode and CountyCode to numbers
df['StateCode'] = pd.to_numeric(df['StateCode'], errors='coerce').fillna(0)
df['CountyCode'] = pd.to_numeric(df['CountyCode'], errors='coerce').fillna(0)

#Display the shape of the data
display(df.shape)
#Display the dataypes of the columns
#display(df.dtypes)
#Display all of the data
#display(df)
df.set_index(['CountyCode', 'StateCode'], inplace = True) #Year was here also- County&StateName?
display(df)

#https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/weather.py

df.head()


(5724, 16)

,,Year,medianFamilyIncome,medianHomeVal,medianHouseIncome,medianHouseholdCosts,medianNonFamIncome,medianRealEstateTax,medianWorkerIncome,population,totalHouses,workers,medianAge,medianIncome,povertyPop
CountyCode,StateCode,,,,,,,,,,,,,,
3,1,2011-01-01,66145,170300,50900,933,33795,557,30104,186717,54247,93840,42,74460,8192
15,1,2011-01-01,49603,99100,39037,605,21414,395,23301,117797,30699,54845,39,44722,10023
43,1,2011-01-01,53091,112000,40054,677,15370,311,27788,80536,23942,36309,41,31954,4921
49,1,2011-01-01,40978,97100,36541,571,16159,340,24351,71375,21897,33281,39,27139,3544
51,1,2011-01-01,64952,148900,57405,860,34993,399,30716,80162,22520,38421,37,28469,5851
55,1,2011-01-01,45075,103600,33313,628,18277,441,24554,104303,29577,43441,39,40443,8618
69,1,2011-01-01,50657,123600,40336,656,24819,418,25041,102369,25868,49037,38,39118,7645
73,1,2011-01-01,56260,141100,41976,877,26384,873,27467,658931,163753,320232,36,255059,51027
77,1,2011-01-01,53654,109400,40121,628,23340,510,23701,92781,28087,43388,40,39331,7501


,,Year,medianFamilyIncome,medianHomeVal,medianHouseIncome,medianHouseholdCosts,medianNonFamIncome,medianRealEstateTax,medianWorkerIncome,population,totalHouses,workers,medianAge,medianIncome,povertyPop
CountyCode,StateCode,,,,,,,,,,,,,,
3,1,2011-01-01,66145,170300,50900,933,33795,557,30104,186717,54247,93840,42,74460,8192
15,1,2011-01-01,49603,99100,39037,605,21414,395,23301,117797,30699,54845,39,44722,10023
43,1,2011-01-01,53091,112000,40054,677,15370,311,27788,80536,23942,36309,41,31954,4921
49,1,2011-01-01,40978,97100,36541,571,16159,340,24351,71375,21897,33281,39,27139,3544
51,1,2011-01-01,64952,148900,57405,860,34993,399,30716,80162,22520,38421,37,28469,5851
